# Food-101 EfficientNet Training

This notebook trains a food dish classifier using EfficientNet-B0 fine-tuned on the Food-101 dataset.

In [ ]:
# Install dependencies
!pip install timm

In [ ]:
import os
import shutil
import random
import time
import json
from pathlib import Path
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import timm
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Download Dataset
if not os.path.exists('food-101.tar.gz'):
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("Downloaded food-101.tar.gz")

if not os.path.exists('food-101'):
    !tar xzf food-101.tar.gz
    print("Extracted food-101.tar.gz")

In [ ]:
# Rebuild Dataset Structure
original_data_dir = Path('food-101')
images_dir = original_data_dir / 'images'
meta_dir = original_data_dir / 'meta'

train_meta_file = meta_dir / 'train.txt'
test_meta_file = meta_dir / 'test.txt'

rebuilt_dir = Path('food101_rebuilt')
train_dir = rebuilt_dir / 'train'
val_dir = rebuilt_dir / 'val'

if not rebuilt_dir.exists():
    print("Rebuilding dataset structure...")
    
    # Create directories
    for split_dir in [train_dir, val_dir]:
        split_dir.mkdir(parents=True, exist_ok=True)
        
    # Helper to process files
    def process_split(meta_file, destination):
        with open(meta_file, 'r') as f:
            image_paths = [line.strip() for line in f.readlines()]
            
        for img_path_str in tqdm(image_paths, desc=f"Processing {destination.name}"):
            class_name = img_path_str.split('/')[0]
            
            # Source file
            src_file = images_dir / f"{img_path_str}.jpg"
            
            # Dest directory
            dest_class_dir = destination / class_name
            dest_class_dir.mkdir(exist_ok=True)
            
            # Copy
            shutil.copy(src_file, dest_class_dir / f"{src_file.name}")

    process_split(train_meta_file, train_dir)
    process_split(test_meta_file, val_dir)
    
    print("Dataset rebuild complete.")
else:
    print("Dataset already rebuilt.")

In [ ]:
# Transforms
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Dataloaders
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

class_names = train_dataset.classes
print(f"Number of classes: {len(class_names)}")

In [ ]:
# Model Setup
model = timm.create_model('efficientnet_b0', pretrained=True)

# Freeze backbone
for param in model.parameters():
    param.requires_grad = False

# Replace classifier head
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, len(class_names))

model = model.to(device)

In [ ]:
# Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
# Training Loop
num_epochs = 6
best_acc = 0.0
save_path = "food101_efficientnet.pt"

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)

    # Training Phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += torch.sum(preds == labels.data)
        total_train += labels.size(0)

    scheduler.step()
    
    epoch_loss = running_loss / total_train
    epoch_acc = correct_train.double() / total_train
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    # Validation Phase
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_val += torch.sum(preds == labels.data)
            total_val += labels.size(0)

    val_epoch_loss = val_loss / total_val
    val_epoch_acc = correct_val.double() / total_val
    print(f"Val Loss: {val_epoch_loss:.4f} Acc: {val_epoch_acc:.4f}")

    # Save best model
    if val_epoch_acc > best_acc:
        best_acc = val_epoch_acc
        torch.save(model.state_dict(), save_path)
        print(f"Saved best model weights to {save_path}")

print(f"Best Validation Accuracy: {best_acc:.4f}")

In [ ]:
# Inference Test
def predict_image(image_path, model, transform, class_names):
    image = Image.open(image_path).convert("RGB")
    data = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(data)
        _, pred = torch.max(output, 1)
        prob = torch.nn.functional.softmax(output, dim=1)[0] * 100
    
    print(f"Predicted: {class_names[pred.item()]} ({prob[pred.item()]:.2f}%)")
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Test on a random validation image
random_class = random.choice(class_names)
random_img_name = random.choice(os.listdir(val_dir / random_class))
test_image_path = val_dir / random_class / random_img_name

print(f"Testing on image: {test_image_path}")
print(f"True Label: {random_class}")
predict_image(test_image_path, model, val_transform, class_names)

In [ ]:
# Download Model
if os.path.exists("food101_efficientnet.pt"):
    files.download("food101_efficientnet.pt")
    print("Downloading model file...")
else:
    print("Model file not found.")